# Streaming
This notebook covers functionality related to streaming.

For more information, see:

- [Streaming with LCEL](https://js.langchain.com/docs/expression_language/interface#stream)
- [Streaming for Agents](https://js.langchain.com/docs/modules/agents/how_to/streaming)

## Basic Streaming

In [1]:
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";

In [2]:
const prompt = ChatPromptTemplate.fromTemplate("Tell me a joke about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const parser = new StringOutputParser();
const chain = prompt.pipe(model).pipe(parser);

In [3]:
for await (const step of await chain.stream({
  topic: "bears",
})) {
  console.log(step);
}


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



## Streaming with RunnableParallel

In [4]:
import { RunnableParallel } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromTemplate("Tell me a joke about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const parser = new StringOutputParser();
const chain1 = prompt.pipe(model).pipe(parser);

In [5]:
const prompt = ChatPromptTemplate.fromTemplate("Write me a poem about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const parser = new StringOutputParser();
const chain2 = prompt.pipe(model).pipe(parser);

In [6]:
const parallelChain = new RunnableParallel({
  steps: {
    joke: chain1,
    poem: chain2
  }
});

for await (const step of await parallelChain.stream({
  topic: "bears",
})) {
  console.log(step);
}

{ poem: "" }
{ poem: "In" }
{ poem: " the" }
{ poem: " heart" }
{ poem: " of" }
{ poem: " the" }
{ poem: " wild" }
{ poem: "," }
{ poem: " where" }
{ poem: " nature" }
{ joke: "" }
{ joke: "Why" }
{ joke: " don" }
{ joke: "'t" }
{ joke: " bears" }
{ joke: " wear" }
{ joke: " shoes" }
{ poem: " thr" }
{ poem: "ives" }
{ poem: ",\n" }
{ poem: "Ro" }
{ poem: "ams" }
{ poem: " a" }
{ joke: "?\n\n" }
{ joke: "Because" }
{ joke: " they" }
{ joke: " have" }
{ joke: " bear" }
{ joke: " feet" }
{ joke: "!" }
{ joke: "" }
{ poem: " majestic" }
{ poem: " creature" }
{ poem: "," }
{ poem: " with" }
{ poem: " strength" }
{ poem: " and" }
{ poem: " drive" }
{ poem: ".\n" }
{ poem: "With" }
{ poem: " fur" }
{ poem: " of" }
{ poem: " gold" }
{ poem: "," }
{ poem: " or" }
{ poem: " black" }
{ poem: " as" }
{ poem: " night" }
{ poem: ",\n" }
{ poem: "B" }
{ poem: "ears" }
{ poem: " command" }
{ poem: " the" }
{ poem: " forest" }
{ poem: "," }
{ poem: " their" }
{ poem: " realm" }
{ poem: " of" }
{ poem:

In [7]:
let result = {};
for await (const step of await parallelChain.stream({
  topic: "bears",
})) {
  for (const [k, v] of Object.entries(step)) {
    result[k] = result[k] || "";
    result[k] += v;
  }
  console.log(result);
}

{ poem: "" }
{ poem: "", joke: "" }
{ poem: "", joke: "Why" }
{ poem: "", joke: "Why don" }
{ poem: "", joke: "Why don't" }
{ poem: "In", joke: "Why don't" }
{ poem: "In", joke: "Why don't bears" }
{ poem: "In the", joke: "Why don't bears" }
{ poem: "In the", joke: "Why don't bears wear" }
{ poem: "In the heart", joke: "Why don't bears wear" }
{ poem: "In the heart", joke: "Why don't bears wear shoes" }
{ poem: "In the heart of", joke: "Why don't bears wear shoes" }
{ poem: "In the heart of the", joke: "Why don't bears wear shoes" }
{
  poem: "In the heart of the wild",
  joke: "Why don't bears wear shoes"
}
{
  poem: "In the heart of the wild,",
  joke: "Why don't bears wear shoes"
}
{
  poem: "In the heart of the wild, where",
  joke: "Why don't bears wear shoes"
}
{
  poem: "In the heart of the wild, where nature",
  joke: "Why don't bears wear shoes"
}
{
  poem: "In the heart of the wild, where nature thr",
  joke: "Why don't bears wear shoes"
}
{
  poem: "In the heart of the wild,

## Stream Log

In [8]:
import { TavilySearchAPIRetriever } from "@langchain/community/retrievers/tavily_search_api";
import { formatDocumentsAsString } from "langchain/util/document";
import { RunnablePassthrough } from "@langchain/core/runnables";

const retriever = new TavilySearchAPIRetriever();

const prompt = ChatPromptTemplate.fromTemplate(`Answer the question based only on the context provided:

Context: {context}

Question: {question}`);

const question = "what is langsmith"

const chain = prompt.pipe(model).pipe(parser);

const retrievalChain = RunnablePassthrough.assign({
  context: async (i: { question: string }) =>
  formatDocumentsAsString(await retriever.withConfig({ runName: "Docs" }).invoke(question))
}).withConfig({ runName: "passthrough" }).pipe(chain);

for await (const step of await retrievalChain.streamLog({
  question,
})) {
  console.log(step);
}

RunLogPatch {
  ops: [
    {
      op: "replace",
      path: "",
      value: {
        id: "3eab58e9-e4e7-446b-b4b0-bc14ae4f3ca9",
        streamed_output: [],
        final_output: undefined,
        logs: {}
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/passthrough",
      value: {
        id: "7e1c773d-743c-43b7-98cd-28f6e2ecd094",
        name: "passthrough",
        type: "chain",
        tags: [ "seq:step:1" ],
        metadata: {},
        start_time: "2024-01-08T20:55:49.927Z",
        streamed_output_str: [],
        final_output: undefined,
        end_time: undefined
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/RunnableMap",
      value: {
        id: "9bef378d-b676-4043-a568-19604435e1eb",
        name: "RunnableMap",
        type: "chain",
        tags: [],
        metadata: {},
        start_time: "2024-01-08T20:55:50.050Z",
        streamed_output_str: [],
        final_output: undefined,
  

In [9]:
for await (const step of await retrievalChain.streamLog(
  {
    question,
  },
  {},
  {
    includeNames: ["Docs"]
  }
)) {
  console.log(step);
}

RunLogPatch {
  ops: [
    {
      op: "replace",
      path: "",
      value: {
        id: "dda1bfc6-a13d-4a1c-8735-ebae341a4959",
        streamed_output: [],
        final_output: undefined,
        logs: {}
      }
    }
  ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: "" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: "Lang" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: "Smith" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " is" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " a" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " platform" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " that" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " offers" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " features" } ]
}
RunL

## Agents

### Stream Actions

In [10]:
import { pull } from "langchain/hub";
import { createOpenAIFunctionsAgent, AgentExecutor } from "langchain/agents";
import { ChatOpenAI } from "@langchain/openai";
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

// Get the prompt to use - you can modify this!
const prompt = await pull<ChatPromptTemplate>("hwchase17/openai-functions-agent");

const search = new TavilySearchResults();
const tools = [search];

const llm = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});

const agent = await createOpenAIFunctionsAgent({
  llm,
  tools,
  prompt,
});
const agentExecutor = new AgentExecutor({ agent, tools });

In [11]:
for await (const chunk of await agentExecutor.stream({
  input: "what is the weather in SF and then LA"
})) {
  console.log(chunk);
  console.log("---------")
}

{
  intermediateSteps: [
    {
      action: {
        tool: "tavily_search_results_json",
        toolInput: { input: "weather in San Francisco" },
        log: 'Invoking "tavily_search_results_json" with {\n' +
          '  "input": "weather in San Francisco"\n' +
          "}\n",
        messageLog: [ [AIMessage] ]
      },
      observation: '[{"title":"Weather in San Francisco in January 2024","url":"https://world-weather.info/forecast/usa/'... 2004 more characters
    }
  ]
}
---------
{
  intermediateSteps: [
    {
      action: {
        tool: "tavily_search_results_json",
        toolInput: { input: "weather in Los Angeles" },
        log: 'Invoking "tavily_search_results_json" with {\n' +
          '  "input": "weather in Los Angeles"\n' +
          "}\n",
        messageLog: [ [AIMessage] ]
      },
      observation: '[{"title":"Weather for Los Angeles | People°s Weather","url":"https://www.peoplesweather.com/weather'... 2355 more characters
    }
  ]
}
---------
{
  output

### Stream Tokens

In [12]:
const llm = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
  streaming: true,
});
const agent = await createOpenAIFunctionsAgent({
  llm,
  tools,
  prompt,
});
const agentExecutor = new AgentExecutor({ agent, tools });

In [13]:
for await (const step of await agentExecutor.streamLog(
  {
    input: "what is the weather in SF"
  },
  {},
  {
    includeNames: ["ChatOpenAI"]
  }
)) {
  console.log(step);
}

RunLogPatch {
  ops: [
    {
      op: "replace",
      path: "",
      value: {
        id: "4a080488-a0eb-4cd9-8351-25865ee8545f",
        streamed_output: [],
        final_output: undefined,
        logs: {}
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI",
      value: {
        id: "ed87f596-e0d3-4a96-8a2d-ccfddac02530",
        name: "ChatOpenAI",
        type: "llm",
        tags: [ "seq:step:3" ],
        metadata: {},
        start_time: "2024-01-08T20:56:13.050Z",
        streamed_output_str: [],
        final_output: undefined,
        end_time: undefined
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI/streamed_output_str/-",
      value: ""
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI/streamed_output_str/-",
      value: ""
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI/streamed_output_str/-",
  